In [ ]:
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
import arch
import datetime as dt

In [ ]:
spx = pd.read_csv('^GSPC.csv', index_col=0)

In [ ]:
spx['daily_chg'] = spx['Adj Close'].pct_change()

Fitting GARCH model

In [ ]:
model=arch.arch_model(spx['daily_chg'][1500:]*100, vol='Garch', p=1, o=1, q=1, dist='skewt')
results=model.fit()
pd.DataFrame(results.params)

Simulating based on estimated student-t GARCH model

In [ ]:
def simulate_market(years, data_to_simulate, p=1, o=1, q=1):
    '''
    Simulated a market using a gjr-garch(1,1) with a skewed t-distribution to draw error term
    
    returns
    
    a dataframe with returns, volatility, error terms and market prices given index 100 at t=0
    '''
    model=arch.arch_model(data_to_simulate, vol='Garch', p=p, o=o, q=q, dist='skewt')
    results=model.fit(disp='off')
    
    #setting horizon
    horizon = 252*years
    market = np.empty((horizon, 1))
    market[0, 0] = 100
    rs = np.random.RandomState(2)
    
    #dist = arch.univariate.SkewStudent(random_state=rs)
    dist = arch.univariate.SkewStudent(rs)
    vol = arch.univariate.GARCH(p=p, o=o, q=q)
    repro_mod = arch.univariate.ConstantMean(data_to_simulate, volatility=vol, distribution=dist)
        
    
    returns=repro_mod.simulate(results.params, horizon)
    
    for i, err in enumerate(returns["data"].values, start=1):
        if i < (horizon):
            market[i, 0] = market[i-1, 0]*(1+err/100)
    returns["Price"]=pd.DataFrame({'Price':market[:, 0]})
    return returns



Plotting

In [ ]:
horizon = 60
returns=simulate_market(horizon, spx['daily_chg'][-252*40:]*100, p=1, q=1)

f, (ax1, ax2 , ax3) = plt.subplots(3, 1, figsize=(15,10))
ax1.plot(returns["Price"])
ax1.set_title('Market')
ax1.set_yscale('log')

ax2.plot(returns["data"])
ax2.set_title('returns')
#ax2.set_yscale('LOG')

ax3.plot(returns["volatility"])
ax3.set_title('Volatility')
#ax3.set_yscale('LOG')

Simulating many markets

In [ ]:
def many_market(nobs,years):
    '''
    this simulated nobs markets with a horizon of years years

    return a dataframe with nobs columns and years*252 rows 
    '''
    data=pd.DataFrame({'n':range(0,horizon)})
    for i in range(0,nobs):
        data["Price" + str(i)]=simulate_market(years)["Price"] 
    return data

Descriptive statistic of simulations

In [ ]:
final_returns=many_market(1000,60).iloc[7560,: ].values
plt.hist(final_returns, bins = 30)
plt.show()

Prospect theory utility function

In [ ]:
def prospect_ut(price_before,price_after,alpha=0.88, beta=0.88, lambdas=2.25):
    '''
    in prospect theory, investors evaluate losses and gains differently. Assumes two different
    domains. Loss domain, where investment with negative returns are evaluted and gain domain for investment with positive return

    price_before:      the price in the begining of the evaluation period
    price_after:       the price in the end of the evulation period
    alpha:             risk aversion in gains
    beta:              risk seekingness in losses
    lambdas:           psycological harm of loss relativ to gains.

    example of price: price before: price of portfolio at 1st of january and price_after the price at 31st of janunary
    
    returns: utility u
    '''
    #absolute change
    x=price_after-price_before
    if x>=0: #if non negative absolute return
        u=x**alpha
    elif x<0:    #if negative absolute return
        u=-lambdas*(-x)**beta
    return u
    

In [ ]:
print(prospect_ut(100,250))

Constant relative risk aversion (CRRA) utility function


In [ ]:
def CRRA_ut():
    return